# Workfile Examples

Let's run a workflow with workflow variable input and download the results locally.

In [1]:
username = 'demoadmin'
password = 'password'
host = "10.10.0.204"
port = "8080"

test_workflow_name = "WineData"
test_workspace_name = "APITests"

In [2]:
import api as AlpineAPI
from pprint import pprint

In [3]:
session = AlpineAPI.Alpine(host, port, username, password)

## Finding our workflow

In [7]:
test_workspace_id = session.workspace.get_id(test_workspace_name)
all_workfiles = session.workfile.get_all(test_workspace_id)
pprint(all_workfiles, depth = 2)

[{u'associated_worklets': [],
  u'comment_count': 0,
  u'complete_json': True,
  u'description': u'',
  u'entity_subtype': u'alpine',
  u'entity_type': u'workfile',
  u'file_name': u'Test workflow',
  u'file_type': u'work_flow',
  u'id': 7,
  u'is_deleted': None,
  u'latest_version_id': None,
  u'link_url': None,
  u'owner': {...},
  u'recent_comments': [],
  u'status': u'idle',
  u'tags': [],
  u'user_modified_at': u'2016-12-20T00:41:27Z',
  u'version_info': {...},
  u'workspace': {...}},
 {u'associated_worklets': [],
  u'comment_count': 0,
  u'complete_json': True,
  u'description': u'',
  u'entity_subtype': u'alpine',
  u'entity_type': u'workfile',
  u'file_name': u'WineData',
  u'file_type': u'work_flow',
  u'id': 375,
  u'is_deleted': None,
  u'latest_version_id': None,
  u'link_url': None,
  u'owner': {...},
  u'recent_comments': [],
  u'status': u'idle',
  u'tags': [],
  u'user_modified_at': u'2017-02-07T22:28:29Z',
  u'version_info': {...},
  u'workspace': {...}}]


In [8]:
test_workflow_id = session.workfile.get_id(test_workflow_name, test_workspace_id)
single_workfile_info = session.workfile.get_data(test_workflow_id)
pprint(single_workfile_info, depth = 2)

{u'response': {u'associated_worklets': [],
               u'comment_count': 0,
               u'complete_json': True,
               u'dataset_ids': [],
               u'description': u'',
               u'entity_subtype': u'alpine',
               u'entity_type': u'workfile',
               u'execution_locations': [...],
               u'file_name': u'WineData',
               u'file_type': u'work_flow',
               u'id': 375,
               u'is_deleted': None,
               u'latest_version_id': None,
               u'link_url': None,
               u'owner': {...},
               u'recent_comments': [],
               u'status': u'idle',
               u'tags': [],
               u'user_modified_at': u'2017-02-07T22:28:29Z',
               u'version_info': {...},
               u'workspace': {...}}}


## Running a workflow

In [9]:
process_id = session.workfile.run(test_workflow_id)

In [10]:
session.workfile.query_status(process_id)

'WORKING'

In [11]:
session.workfile.wait_until_finished(process_id, verbose=True, query_time=5, timeout=100)

Workflow in progress for ~0.15 seconds.

Workflow in progress for ~5.23 seconds.

Workflow in progress for ~10.30 seconds.

Workflow in progress for ~15.39 seconds.

Workflow in progress for ~20.48 seconds.

Workflow in progress for ~25.58 seconds.

Workflow in progress for ~30.67 seconds.

'FINISHED'

## Downloading the flow results

In [12]:
flow_results = session.workfile.download_results(test_workflow_id, process_id)

In [13]:
operator_data = session.workfile.find_operator('Row Filter', flow_results);
pprint(operator_data, depth = 2)

{u'isGenerateReport': True,
 u'node_meta_info': [[...], [...], [...], [...], [...]],
 u'out_id': 1487088274100.573,
 u'out_title': u'Row Filter',
 u'visualData': {u'columnTypes': [...],
                 u'columns': [...],
                 u'errorMessages': [...],
                 u'fixedNumberOfColumns': 0,
                 u'items': [...]},
 u'visualType': 0}


In [14]:
pprint(session.workfile.get_flow_metadata(flow_results))

{u'endTime': u'2017-02-14T08:04:49.028-0800',
 u'executeUser': u'665',
 u'noOfError': 0,
 u'noOfNodesProcessed': 3,
 u'processId': u'71f64a41-9bed-4a8c-9b7e-5f527a6b3275',
 u'startTime': u'2017-02-14T08:04:06.262-0800',
 u'status': u'SUCCESS',
 u'workflowId': u'375',
 u'workflowName': u'WineData'}


## Using workflow variables

After properly defining a workflow variable in Alpine UI, a user can inject different values into a workflow. In this workflow I've defined a workflow variable that is being used in a Row Filter operator.

In [15]:
variables = [{"name": "@row_filter", "value": "13"}]
process_id = session.workfile.run(test_workflow_id, variables=variables)
session.workfile.wait_until_finished(process_id, verbose=True, query_time=5, timeout=100)

Workflow in progress for ~0.12 seconds.

Workflow in progress for ~5.20 seconds.

Workflow in progress for ~10.27 seconds.

Workflow in progress for ~15.35 seconds.

Workflow in progress for ~20.43 seconds.

Workflow in progress for ~25.51 seconds.

Workflow in progress for ~30.68 seconds.

Workflow in progress for ~35.85 seconds.

Workflow in progress for ~41.17 seconds.

'FINISHED'

In [16]:
flow_results = session.workfile.download_results(test_workflow_id, process_id)
ss_results = session.workfile.find_operator("Summary Statistics", flow_results)
pprint(ss_results['visualData']['items'][0]['count'])

u'86'
